In [362]:
import psycopg2
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score



from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [363]:
conn = psycopg2.connect(
    host="localhost",
    database="deliv3", 
    user="postgres",
    password="m")

df_country = pd.read_sql_query("select * from country;", conn)
df_devidx = pd.read_sql_query("select * from index_dev", conn)

In [364]:
df_country = pd.read_sql_query("select * from country;", conn)

for yr in ['yr_2005', 'yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011', 'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017', 'yr_2018', 'yr_2019', 'yr_2020']:
    df_country[yr] = df_country[yr].apply(lambda x: '%.3f' % x)

df_country = pd.read_csv('./spreadsheets/HNP_StatsCountry.csv')
df_country.dropna(inplace=True)
df_country = df_country.loc[df_country['countrycode'].isin(df_devidx['countrycode'])]

df_country.reset_index(inplace=True)

In [365]:
df_devidx = pd.read_sql_query("select * from index_dev", conn)
df_devidx.reset_index(drop=True, inplace=True)

df_devidx

,countryname,countrycode,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020
0,Afghanistan,AFG,0.16,0.16,0.16,0.16,0.16,0.18,0.18,0.18,0.18,0.18,0.25,0.25,0.25,0.25,0.25,0.25
1,Albania,ALB,0.46,0.46,0.51,0.51,0.51,0.53,0.53,0.53,0.53,0.53,0.59,0.59,0.59,0.59,0.59,0.59
2,Algeria,DZA,0.39,0.39,0.40,0.40,0.40,0.41,0.41,0.41,0.41,0.41,0.45,0.45,0.45,0.45,0.45,0.45
3,Angola,AGO,0.22,0.22,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.27,0.27,0.27,0.27,0.27,0.27
4,Argentina,ARG,0.61,0.61,0.63,0.63,0.63,0.65,0.65,0.65,0.65,0.65,0.70,0.70,0.70,0.70,0.70,0.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,Venezuela,VEN,0.50,0.50,0.55,0.55,0.55,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56
160,Vietnam,VNM,0.43,0.43,0.43,0.43,0.43,0.45,0.45,0.45,0.45,0.45,0.48,0.48,0.48,0.48,0.48,0.48
161,Yemen,YEM,0.28,0.28,0.30,0.30,0.30,0.32,0.32,0.32,0.32,0.32,0.30,0.30,0.30,0.30,0.30,0.30
162,Zambia,ZMB,0.22,0.22,0.21,0.21,0.21,0.24,0.24,0.24,0.24,0.24,0.31,0.31,0.31,0.31,0.31,0.31


In [366]:
df_qol = pd.read_sql_query("select * from quality_dim", conn)

df_qol.drop('qualitykey', axis=1, inplace=True)

df_qol[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']] = df_qol[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']].apply(pd.to_numeric, errors='coerce')


df_qol = df_qol[~df_qol[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']].eq(0).sum(1).ge(3)]

df_qol.head()

,series_name,series_code,country_name,country_code,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020
4,People with basic handwashing facilities inclu...,SH.STA.HYGN.ZS,Niger,NER,7.123087,8.208276,9.293511,10.378542,11.463872,12.549247,13.634435,14.719907,15.805423,16.894199,17.986401,19.081334,20.179142,21.279151,22.381482,23.485846
9,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,Canada,CAN,99.234644,99.234620,99.235497,99.236077,99.236355,99.236349,99.235894,99.234326,99.232686,99.230985,99.229221,99.227395,99.225538,99.223631,99.221650,99.221811
10,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,United States,USA,98.853687,98.860837,98.867945,98.875013,98.882040,98.978952,99.074887,99.169688,99.263379,99.355881,99.447158,99.537139,99.625816,99.713121,99.799044,99.883527
11,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,Brazil,BRA,95.084137,95.384183,95.677625,95.964568,96.244799,96.518797,96.787980,97.050980,97.307676,97.558582,97.803568,97.955839,98.450955,98.679495,98.935544,99.320853
12,People using at least basic drinking water ser...,SH.H2O.BASW.ZS,Mexico,MEX,93.096689,93.710997,94.320322,94.908091,95.367103,95.821334,96.271203,96.716444,97.157255,97.593626,98.025444,98.452865,98.875781,99.269132,99.477264,99.679568


In [367]:
basicSan = df_qol.loc[df_qol['series_name'].str.contains('People using at least basic sanitation services \(% of'), 'country_code':]
keep_same = {'country_code'}
basicSan.columns = ['{}{}'.format('' if c in keep_same else 'BasicSan_', c) 
                    for c in basicSan.columns]

basicSan

,country_code,BasicSan_yr_2005,BasicSan_yr_2006,BasicSan_yr_2007,BasicSan_yr_2008,BasicSan_yr_2009,BasicSan_yr_2010,BasicSan_yr_2011,BasicSan_yr_2012,BasicSan_yr_2013,BasicSan_yr_2014,BasicSan_yr_2015,BasicSan_yr_2016,BasicSan_yr_2017,BasicSan_yr_2018,BasicSan_yr_2019,BasicSan_yr_2020
27,CAN,99.805195,99.750338,99.695954,99.641457,99.586846,99.532131,99.477216,99.421719,99.366191,99.310644,99.255071,99.199477,99.143884,99.088289,99.032681,99.032862
28,USA,99.881787,99.881540,99.881295,99.881055,99.880815,99.860766,99.841033,99.821629,99.802569,99.783861,99.765524,99.747554,99.729968,99.712774,99.695994,99.679619
29,BRA,77.564792,78.432906,79.295976,80.154083,81.006527,81.854204,82.700581,83.542226,84.378518,85.210642,86.037962,86.860243,87.677958,88.490545,89.298796,90.078471
30,MEX,81.273778,82.078307,82.872931,83.658304,84.434192,85.201028,85.959710,86.709804,87.451783,88.185732,88.911484,89.629447,90.339476,91.041845,91.736661,92.424063
31,NER,7.901512,8.401596,8.893620,9.377293,9.853170,10.320974,10.780380,11.232013,11.675560,12.116385,12.555830,12.993835,13.431859,13.869850,14.309386,14.751306
32,SYR,91.088288,91.024424,90.956335,90.883884,90.807035,90.725976,90.577673,90.439936,90.312975,90.196737,90.131757,90.059184,89.979025,89.891321,89.796115,89.693480
33,ZAF,64.015806,65.048130,66.071659,67.086321,68.091224,69.086684,70.072349,71.048142,72.013582,72.968804,73.913539,74.847618,75.770868,76.683188,77.584480,78.474611
34,COD,22.052001,21.550233,21.056586,20.571029,20.093689,19.624513,19.163642,18.711002,18.266617,17.830575,17.402839,16.983426,16.572272,16.169464,15.774931,15.388672
35,THA,93.837817,94.137536,94.442578,94.753025,95.069096,95.390776,95.725177,96.064622,96.407527,96.753882,97.103707,97.455331,97.790907,98.125218,98.420525,98.713119


In [368]:
unemployment = df_qol.loc[df_qol['series_name'].str.contains('Unemployment, total'), 'country_code':]
keep_same = {'country_code'}
unemployment.columns = ['{}{}'.format('' if c in keep_same else 'UnEmployed_', c) 
                    for c in unemployment.columns]

df_quality = pd.merge(unemployment, basicSan, on='country_code', how='inner')
df_quality.rename(columns={'country_code': 'countrycode'}, inplace=True)
df_quality

,countrycode,UnEmployed_yr_2005,UnEmployed_yr_2006,UnEmployed_yr_2007,UnEmployed_yr_2008,UnEmployed_yr_2009,UnEmployed_yr_2010,UnEmployed_yr_2011,UnEmployed_yr_2012,UnEmployed_yr_2013,...,BasicSan_yr_2011,BasicSan_yr_2012,BasicSan_yr_2013,BasicSan_yr_2014,BasicSan_yr_2015,BasicSan_yr_2016,BasicSan_yr_2017,BasicSan_yr_2018,BasicSan_yr_2019,BasicSan_yr_2020
0,CAN,6.760000,6.320,6.040000,6.140,8.340,8.060,7.510000,7.290,7.070000,...,99.477216,99.421719,99.366191,99.310644,99.255071,99.199477,99.143884,99.088289,99.032681,99.032862
1,USA,5.080000,4.620,4.620000,5.780,9.250,9.630,8.950000,8.070,7.370000,...,99.841033,99.821629,99.802569,99.783861,99.765524,99.747554,99.729968,99.712774,99.695994,99.679619
2,BRA,9.570000,8.640,8.330000,7.340,8.520,7.274,6.920000,7.190,6.980000,...,82.700581,83.542226,84.378518,85.210642,86.037962,86.860243,87.677958,88.490545,89.298796,90.078471
3,MEX,3.560000,3.570,3.630000,3.870,5.360,5.300,5.170000,4.890,4.910000,...,85.959710,86.709804,87.451783,88.185732,88.911484,89.629447,90.339476,91.041845,91.736661,92.424063
4,NER,3.100000,2.649,2.203000,1.710,1.267,0.779,0.320000,2.180,1.369000,...,10.780380,11.232013,11.675560,12.116385,12.555830,12.993835,13.431859,13.869850,14.309386,14.751306
5,SYR,8.853000,8.170,8.420000,10.940,8.140,8.610,8.697000,8.808,8.826000,...,90.577673,90.439936,90.312975,90.196737,90.131757,90.059184,89.979025,89.891321,89.796115,89.693480
6,ZAF,29.120001,28.340,26.540001,22.410,23.520,24.680,24.639999,24.730,24.559999,...,70.072349,71.048142,72.013582,72.968804,73.913539,74.847618,75.770868,76.683188,77.584480,78.474611
7,COD,2.850000,3.083,3.313000,3.547,3.793,4.014,4.252000,4.490,4.476000,...,19.163642,18.711002,18.266617,17.830575,17.402839,16.983426,16.572272,16.169464,15.774931,15.388672
8,THA,1.350000,1.220,1.180000,1.180,0.949,0.620,0.660000,0.580,0.250000,...,95.725177,96.064622,96.407527,96.753882,97.103707,97.455331,97.790907,98.125218,98.420525,98.713119


In [372]:
df = pd.merge(df_country[['countrycode', 'Region', 'incomegroup']], df_devidx, on='countrycode', how='outer')
df = pd.merge(df, df_quality, on='countrycode', how='outer')
df.fillna(0, inplace=True)

region_1hot = pd.get_dummies(df['Region'])
df.drop('Region', axis=1, inplace=True)
df = df.join(region_1hot)

df = df.loc[df['incomegroup'] != 0]

X, y = df.loc[:, ~df.columns.isin(['countryname', 'countrycode','incomegroup'])], df['incomegroup']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

df

,countrycode,incomegroup,countryname,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,...,BasicSan_yr_2019,BasicSan_yr_2020,0,East Asia & Pacific,Europe & Central Asia,Latin America & Caribbean,Middle East & North Africa,North America,South Asia,Sub-Saharan Africa
0,AFG,Low income,Afghanistan,0.16,0.16,0.16,0.16,0.16,0.18,0.18,...,0.00000,0.000000,0,0,0,0,0,0,1,0
1,AGO,Lower middle income,Angola,0.22,0.22,0.24,0.24,0.24,0.24,0.24,...,0.00000,0.000000,0,0,0,0,0,0,0,1
2,ALB,Upper middle income,Albania,0.46,0.46,0.51,0.51,0.51,0.53,0.53,...,0.00000,0.000000,0,0,1,0,0,0,0,0
3,ARE,High income,United Arab Emirates,0.51,0.51,0.53,0.53,0.53,0.57,0.57,...,0.00000,0.000000,0,0,0,0,1,0,0,0
4,ARG,Upper middle income,Argentina,0.61,0.61,0.63,0.63,0.63,0.65,0.65,...,0.00000,0.000000,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,VNM,Lower middle income,Vietnam,0.43,0.43,0.43,0.43,0.43,0.45,0.45,...,0.00000,0.000000,0,1,0,0,0,0,0,0
158,YEM,Low income,Yemen,0.28,0.28,0.30,0.30,0.30,0.32,0.32,...,0.00000,0.000000,0,0,0,0,1,0,0,0
159,ZAF,Upper middle income,South Africa,0.33,0.33,0.34,0.34,0.34,0.35,0.35,...,77.58448,78.474611,0,0,0,0,0,0,0,1
160,ZMB,Lower middle income,Zambia,0.22,0.22,0.21,0.21,0.21,0.24,0.24,...,0.00000,0.000000,0,0,0,0,0,0,0,1


In [373]:
###############################################
######    DecisionTree Classification    ######
###############################################
clf_dt = DecisionTreeClassifier(random_state=0)
clf_dt.fit(X_train, y_train)
pred_dt = clf_dt.predict(X_test)

print("DecisionTree Accuracy:  ", accuracy_score(y_test, pred_dt))
print("DecisionTree Precision: ", precision_score(y_test, pred_dt, average='macro'))
print("DecisionTree Recall:    ", recall_score(y_test, pred_dt, average='macro'))

DecisionTree Accuracy:   0.6481481481481481
DecisionTree Precision:  0.6709150326797386
DecisionTree Recall:     0.6702205882352942


In [374]:
################################################
######    GradientBoost Classification    ######
################################################
clf_gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf_gb.fit(X_train, y_train)
pred_gb = clf_gb.predict(X_test)

print("GradientBoost Accuracy:  ", accuracy_score(y_test, pred_gb))
print("GradientBoost Precision: ", precision_score(y_test, pred_gb, average='weighted'))
print("GradientBoost Recall:    ", recall_score(y_test, pred_gb, average='macro'))

GradientBoost Accuracy:   0.7777777777777778
GradientBoost Precision:  0.7818930041152263
GradientBoost Recall:     0.78968837535014


In [376]:
###############################################
######    RandomForest Classification    ######
###############################################
clf_rf = RandomForestClassifier(max_depth=2, random_state=0)
clf_rf.fit(X_train, y_train)
pred_rf = clf_rf.predict(X_test)

print("RandomForest Accuracy:  ", accuracy_score(y_test, pred_rf))
print("RandomForest Precision: ", precision_score(y_test, pred_rf, average='macro'))
print("RandomForest Recall:    ", recall_score(y_test, pred_rf, average='macro'))

RandomForest Accuracy:   0.7407407407407407
RandomForest Precision:  0.797381072874494
RandomForest Recall:     0.7437324929971989


<hr style="border:1px solid gray"> </hr>

## Bonus: Detecting Outliers


#### Quality of Life Outliers

In [377]:
osvm = OneClassSVM(kernel='rbf', nu=0.03)
res_osvm = osvm.fit(df_qol.select_dtypes(include=[np.number])).predict(df_qol.select_dtypes(include=[np.number]))
df_qol['isOutlier'] = np.where(res_osvm==-1, 'outlier', '')
df_qol.loc[df_qol['isOutlier']=='outlier']

,series_name,series_code,country_name,country_code,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,...,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,isOutlier
21,People using safely managed drinking water ser...,SH.H2O.SMDW.ZS,Mexico,MEX,40.359489,40.564057,40.766416,40.966696,41.164826,41.360894,...,41.747264,41.937538,42.125896,42.312289,42.496783,42.679333,42.849122,42.938961,43.026285,outlier
53,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,Thailand,THA,1.350000,1.220000,1.180000,1.180000,0.949000,0.620000,...,0.580000,0.250000,0.580000,0.600000,0.690000,0.830000,0.770000,0.720000,1.100000,outlier
71,"Unemployment, female (% of female labor force)",SL.UEM.TOTL.FE.ZS,Thailand,THA,1.210000,1.122000,1.066000,1.040000,0.875000,0.620000,...,0.531000,0.239000,0.564000,0.611000,0.712000,0.841000,0.748000,0.736000,1.089000,outlier
106,People using safely managed drinking water ser...,SH.H2O.SMDW.RU.ZS,"Congo, Dem. Rep.",COD,0.598406,0.621899,0.645845,0.670242,0.695090,0.720390,...,0.772345,0.799000,0.826107,0.853665,0.881675,0.910136,0.939049,0.968414,0.998230,outlier
123,People using at least basic sanitation service...,SH.STA.BASS.RU.ZS,South Africa,ZAF,51.282069,53.133427,55.005774,56.899109,58.813432,60.748743,...,64.682332,66.680608,68.699873,70.740127,72.801368,74.883598,76.986817,79.111024,81.256219,outlier
147,People using safely managed sanitation service...,SH.STA.SMSS.UR.ZS,Mexico,MEX,24.430123,26.587374,28.771553,30.982736,33.221000,35.486423,...,40.099055,42.446418,44.821249,47.223625,49.653623,52.111321,54.596795,57.110123,59.651382,outlier


<hr style="border:1px solid gray"> </hr>

#### Country Development-Index Outliers

In [378]:
osvm = OneClassSVM(kernel='rbf', nu=0.03)
res_osvm = osvm.fit(df_devidx.select_dtypes(include=[np.number])).predict(df_devidx.select_dtypes(include=[np.number]))
df_devidx['isOutlier'] = np.where(res_osvm==-1, 'outlier', '')
df_devidx.loc[df_devidx['isOutlier']=='outlier']

,countryname,countrycode,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,isOutlier
6,Australia,AUS,0.84,0.84,0.85,0.85,0.85,0.87,0.87,0.87,0.87,0.87,0.89,0.89,0.89,0.89,0.89,0.89,outlier
64,Guyana,GUY,0.44,0.44,0.45,0.45,0.45,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,0.39,outlier
69,Iceland,ISL,0.79,0.79,0.81,0.81,0.81,0.83,0.83,0.83,0.83,0.83,0.90,0.90,0.90,0.90,0.90,0.90,outlier
109,Niger,NER,0.12,0.12,0.14,0.14,0.14,0.13,0.13,0.13,0.13,0.13,0.17,0.17,0.17,0.17,0.17,0.17,outlier
122,Qatar,QAT,0.50,0.50,0.50,0.50,0.50,0.52,0.52,0.52,0.52,0.52,0.63,0.63,0.63,0.63,0.63,0.63,outlier


<hr style="border:1px solid gray"> </hr>

#### Population Outliers

In [379]:
df_population = pd.read_sql_query("select * from population_dim;", conn)
df_population.drop('populationkey', axis=1, inplace=True)
df_population[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']] = df_population[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']].apply(pd.to_numeric, errors='coerce')

osvm = OneClassSVM(kernel='rbf', nu=0.03)
res_osvm = osvm.fit(df_population.select_dtypes(include=[np.number])).predict(df_population.select_dtypes(include=[np.number]))
df_population['isOutlier'] = np.where(res_osvm==-1, 'outlier', '')
df_population.loc[df_population['isOutlier']=='outlier']

,series_name,series_code,country_name,country_code,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,...,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,isOutlier
28,Net migration,SM.POP.NETM,United States,USA,0.0,0.0,5428756.0,0.0,0.0,0.0,...,4961716.0,0.0,0.0,0.0,0.0,4774029.0,0.0,0.0,0.0,outlier
29,Net migration,SM.POP.NETM,Mexico,MEX,0.0,0.0,-562404.0,0.0,0.0,0.0,...,-422477.0,0.0,0.0,0.0,0.0,-300000.0,0.0,0.0,0.0,outlier


<hr style="border:1px solid gray"> </hr>

#### Education Outliers

In [380]:
df_education = pd.read_sql_query("select * from education_dim;", conn)
df_education.drop('educationkey', axis=1, inplace=True)
df_education[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']] = df_education[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']].apply(pd.to_numeric, errors='coerce')

osvm = OneClassSVM(kernel='rbf', nu=0.03)
res_osvm = osvm.fit(df_education.select_dtypes(include=[np.number])).predict(df_education.select_dtypes(include=[np.number]))
df_education['isOutlier'] = np.where(res_osvm==-1, 'outlier', '')
df_education.loc[df_education['isOutlier']=='outlier']

,country_name,country_code,series_name,series_code,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,...,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,isOutlier
3,Canada,CAN,"Total net enrolment rate, primary, both sexes (%)",UIS.NERT.1,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,99.45167,99.43231,99.39113,99.58907,99.95587,99.88250,99.78809,0.0,0,outlier
34,Mexico,MEX,"Completion rate, primary education, female (%)",UIS.CR.1.F,0.00000,95.21000,95.21000,95.44000,95.44000,96.72000,...,97.12000,97.12000,97.25000,97.25000,96.31248,96.31248,98.27000,0.0,0,outlier
50,"Congo, Dem. Rep.",COD,"Adult literacy rate, population 15+ years, mal...",SE.ADT.LITR.MA.ZS,0.00000,0.00000,76.91569,0.00000,0.00000,0.00000,...,87.90839,0.00000,0.00000,0.00000,88.52469,0.00000,0.00000,0.0,0,outlier
63,Thailand,THA,"Total net enrolment rate, primary, both sexes (%)",UIS.NERT.1,0.00000,98.91908,98.49881,97.95392,98.08255,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0,outlier
76,Brazil,BRA,"Total net enrolment rate, lower secondary, bot...",UIS.NERT.2,0.00000,0.00000,94.88663,0.00000,98.96988,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0,outlier
82,Brazil,BRA,"Completion rate, primary education, female (%)",UIS.CR.1.F,95.22000,95.75000,95.44000,96.03000,96.35000,0.00000,...,97.11000,97.44000,97.82000,97.97000,95.55000,96.19000,96.44000,0.0,0,outlier
86,South Africa,ZAF,"Adult literacy rate, population 15+ years, mal...",SE.ADT.LITR.MA.ZS,0.00000,0.00000,90.71759,0.00000,94.12146,94.14385,...,94.96372,0.00000,95.28685,95.39653,0.00000,87.67775,0.00000,0.0,0,outlier
99,Syrian Arab Republic,SYR,"Total net enrolment rate, primary, both sexes (%)",UIS.NERT.1,99.19218,99.19777,99.23976,99.26240,99.16314,98.94972,...,98.92896,72.39475,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0,outlier


<hr style="border:1px solid gray"> </hr>

#### Health Outliers

In [381]:
df_health = pd.read_sql_query("select * from health_dim;", conn)
df_health.drop('healthkey', axis=1, inplace=True)
df_health[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']] = df_health[['yr_2005','yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011',
       'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'yr_2017',
       'yr_2018', 'yr_2019', 'yr_2020']].apply(pd.to_numeric, errors='coerce')

osvm = OneClassSVM(kernel='rbf', nu=0.003)
res_osvm = osvm.fit(df_health.select_dtypes(include=[np.number])).predict(df_health.select_dtypes(include=[np.number]))
df_health['isOutlier'] = np.where(res_osvm==-1, 'outlier', '')
df_health.loc[df_health['isOutlier']=='outlier']

,series_name,series_code,country_name,country_code,yr_2005,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,...,yr_2012,yr_2013,yr_2014,yr_2015,yr_2016,yr_2017,yr_2018,yr_2019,yr_2020,isOutlier
75,Number of infant deaths,SH.DTH.IMRT,Mexico,MEX,44987.0,43211.0,41560.0,40080.0,38731.0,37469.0,...,35073.0,33910.0,32728.0,31538.0,30368.0,29178.0,28010.0,26892.0,25833,outlier
78,Number of infant deaths,SH.DTH.IMRT,South Africa,ZAF,53752.0,54092.0,52751.0,49598.0,44926.0,40694.0,...,37180.0,36173.0,35302.0,34249.0,33356.0,32777.0,31810.0,30937.0,30153,outlier
85,Number of under-five deaths,SH.DTH.MORT,Niger,NER,112458.0,108652.0,104876.0,101203.0,97652.0,94332.0,...,89162.0,87289.0,85944.0,84709.0,83815.0,83031.0,82367.0,81614.0,81065,outlier
87,Number of under-five deaths,SH.DTH.MORT,South Africa,ZAF,84721.0,86629.0,83796.0,77985.0,69407.0,60679.0,...,49417.0,47013.0,45130.0,43546.0,42080.0,41205.0,40146.0,38969.0,37807,outlier
88,Number of under-five deaths,SH.DTH.MORT,"Congo, Dem. Rep.",COD,325913.0,324260.0,322285.0,320463.0,318497.0,316388.0,...,311744.0,308923.0,306283.0,303017.0,299716.0,295310.0,291464.0,287617.0,283864,outlier
97,Number of deaths ages 20-24 years,SH.DTH.2024,"Congo, Dem. Rep.",COD,21274.0,21687.0,22171.0,22737.0,23355.0,24005.0,...,25283.0,25909.0,26492.0,27116.0,27854.0,28587.0,29183.0,29933.0,30724,outlier


In [382]:
df['incomegroup'].value_counts()

High income            52
Lower middle income    45
Upper middle income    41
Low income             24
Name: incomegroup, dtype: int64